## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.60,73,24,15.68,few clouds
1,1,Darab,IR,28.7519,54.5444,58.55,43,0,2.51,clear sky
2,2,Longyearbyen,SJ,78.2186,15.6401,12.04,67,40,28.77,scattered clouds
3,3,Semirom,IR,31.4167,51.5667,37.60,76,100,1.23,overcast clouds
4,4,Chokurdakh,RU,70.6333,147.9167,-31.68,99,100,0.58,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.60,73,24,15.68,few clouds
9,9,Hithadhoo,MV,-0.6000,73.0833,81.68,71,97,17.20,overcast clouds
12,12,Maragogi,BR,-9.0122,-35.2225,79.48,74,100,10.89,overcast clouds
19,19,Vila Velha,BR,-20.3297,-40.2925,75.15,78,0,9.22,clear sky
25,25,Hilo,US,19.7297,-155.0900,78.53,65,40,6.91,scattered clouds
26,26,Shimoni,KE,-4.6476,39.3817,78.91,81,23,5.06,few clouds
39,39,Sao Gabriel Da Cachoeira,BR,-0.1303,-67.0892,75.63,90,100,1.59,overcast clouds
42,42,Busselton,AU,-33.6500,115.3333,79.86,43,25,4.76,scattered clouds
54,54,Kahului,US,20.8947,-156.4700,83.46,61,1,19.57,clear sky
55,55,Cockburn Town,TC,21.4612,-71.1419,75.16,73,75,6.91,broken clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                151
City                   151
Country                151
Lat                    151
Lng                    151
Max Temp               151
Humidity               151
Cloudiness             151
Wind Speed             151
Current Description    151
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# This step is not required

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.60,few clouds,-23.1203,-134.9692,
9,Hithadhoo,MV,81.68,overcast clouds,-0.6000,73.0833,
12,Maragogi,BR,79.48,overcast clouds,-9.0122,-35.2225,
19,Vila Velha,BR,75.15,clear sky,-20.3297,-40.2925,
25,Hilo,US,78.53,scattered clouds,19.7297,-155.0900,
26,Shimoni,KE,78.91,few clouds,-4.6476,39.3817,
39,Sao Gabriel Da Cachoeira,BR,75.63,overcast clouds,-0.1303,-67.0892,
42,Busselton,AU,79.86,scattered clouds,-33.6500,115.3333,
54,Kahului,US,83.46,clear sky,20.8947,-156.4700,
55,Cockburn Town,TC,75.16,broken clouds,21.4612,-71.1419,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# check for rows where there is no Hotel Name by replacing empty string with nan
import numpy as np
hotel_df.replace("",np.nan).count()

City                   151
Country                151
Max Temp               151
Current Description    151
Lat                    151
Lng                    151
Hotel Name             146
dtype: int64

In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("",np.nan).dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.60,few clouds,-23.1203,-134.9692,People ThankYou
9,Hithadhoo,MV,81.68,overcast clouds,-0.6000,73.0833,Scoop Guest House
12,Maragogi,BR,79.48,overcast clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
19,Vila Velha,BR,75.15,clear sky,-20.3297,-40.2925,Quality Suites Vila Velha
25,Hilo,US,78.53,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
680,Padang,ID,80.65,overcast clouds,-0.9492,100.3543,Mercure Padang
681,Guarapari,BR,75.09,few clouds,-20.6667,-40.4975,Hotel Porto do Sol
685,Seguela,CI,75.42,clear sky,7.9611,-6.6731,Hotel le Carrefour
686,Cururupu,BR,76.68,scattered clouds,-1.8283,-44.8683,POUSADA TRAVÉS'CIA


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [46]:
# 11a. Add a marker layer for each city to the map. 


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))